In [12]:
import os
import glob
import geopandas as gpd
from asp_plot.jitter import read_positions_rotations_from_file

In [13]:
before1 = "../tests/test_data/jitter/csm-10300100A0D00200.r100.adjusted_state.json"
before2 = "../tests/test_data/jitter/csm-10300100A15E3900.r100.adjusted_state.json"
after1 = "../tests/test_data/jitter/jitter_solved_csm-10300100A0D00200.r100.adjusted_state.json"
after2 = "../tests/test_data/jitter/jitter_solved_csm-10300100A15E3900.r100.adjusted_state.json"

In [14]:
positions, rotations = read_positions_rotations_from_file(before1)

In [7]:
len(rotations)

1790

In [10]:
rotations[0]

array([[-0.02051122,  0.99354149, -0.11160013],
       [-0.90913149, -0.06498039, -0.41140914],
       [-0.41600387,  0.09302069,  0.90459269]])